In [ ]:
!pip install transformers
!pip install emoji
!pip install contractions
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu12

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import numpy as np
dir = '/content/drive/My Drive/1513-project/'

#Preprocess Dataset Used For Fine-tuning

In [ ]:
# Preprocess the data to be used for fine-tuning
import pandas as pd
headers = ['target', 'ids', 'date', 'flag', 'user', 'text']
df = pd.read_csv(dir + 'labeled_tweets.csv', encoding='ISO-8859-1', header=None, names=headers)
print(df.head())

   target         ids                          date      flag  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  @Kenichan I dived many times for the ball. Man...  
3          ElleCTF    my whole body feels itchy and like its on fire   
4           Karoli  @nationwideclass no, it's not behaving at all....  


In [ ]:
print(df)

         target         ids                          date      flag  \
0             0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1             0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2             0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3             0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4             0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
...         ...         ...                           ...       ...   
1599995       4  2193601966  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY   
1599996       4  2193601969  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY   
1599997       4  2193601991  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY   
1599998       4  2193602064  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY   
1599999       4  2193602129  Tue Jun 16 08:40:50 PDT 2009  NO_QUERY   

                    user                                               text  
0        _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww

In [ ]:
import emoji
import re
import contractions
def preprocess_data_cleaning(df):
  # First we lower case the data
  df['text_cleaned'] = df['text'].apply(lambda x: x.lower())
  df['text_cleaned'] = df['text_cleaned'].str.replace(r'(http\S+|www\.\S+)', '', regex=True) # Remove url
  df['text_cleaned'] = df['text_cleaned'].str.replace(r'<.*?>', '', regex=True) # remove html tags
  df['text_cleaned'] = df['text_cleaned'].apply(lambda x: emoji.demojize(x)) # remove emojis
  df['text_cleaned'] = df['text_cleaned'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True) # remove special characters
  df['text_cleaned'] = df['text_cleaned'].apply(lambda x: contractions.fix(x)) # Fix contractions

  label_map = {0:0, 2:1, 4:2}
  df['target'] = df['target'].map(label_map)
  df = df.rename(columns={'target': 'label'})

  return df

In [ ]:
df = preprocess_data_cleaning(df)
print(df.head())

   label         ids                          date      flag             user  \
0      0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY  _TheSpecialOne_   
1      0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY    scotthamilton   
2      0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY         mattycus   
3      0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY          ElleCTF   
4      0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY           Karoli   

                                                text  \
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...   
1  is upset that he can't update his Facebook by ...   
2  @Kenichan I dived many times for the ball. Man...   
3    my whole body feels itchy and like its on fire    
4  @nationwideclass no, it's not behaving at all....   

                                        text_cleaned  
0  switchfoot   awww that is a bummer  you should...  
1  is upset that he cannot update his facebook by...  
2  kenichan

# Tokenize Dataset

In [ ]:
from datasets import Dataset

# Load the pd frame into dataset to allow for batch processing
dataset = Dataset.from_pandas(df)
dataset = dataset.remove_columns(['ids', 'date', 'flag', 'user', 'text'])
print(dataset[0:5])

{'label': [0, 0, 0, 0, 0], 'text_cleaned': ['switchfoot   awww that is a bummer  you shoulda got david carr of third day to do it d', 'is upset that he cannot update his facebook by texting it and might cry as a result  school today also blah', 'kenichan i dived many times for the ball managed to save 50  the rest go out of bounds', 'my whole body feels itchy and like its on fire ', 'nationwideclass no its not behaving at all i am mad why am i here because i cannot see you all over there ']}


In [ ]:
from transformers import AutoTokenizer

# Load the BERTweet tokenizer
tokenizer = AutoTokenizer.from_pretrained('vinai/bertweet-base', use_fast=False)

# Function to tokenize the text in batches
def tokenize(text_batch, tokenizer):
  return tokenizer(
    text_batch['text_cleaned'],
    truncation=True,
    padding='max_length',
    max_length=128,
    return_tensors="pt"
  )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.91M [00:00<?, ?B/s]

In [ ]:
# Tokenize the cleaned text data in batches
tokenized_dataset = dataset.map(lambda x: tokenize(x, tokenizer), batched=True)
tokenized_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
print(tokenized_dataset[0])

Map:   0%|          | 0/1600000 [00:00<?, ? examples/s]

{'label': tensor(0), 'input_ids': tensor([    0, 34493,   996,  2666,  6750,    25,    17,    11, 29697,    14,
        10525,   100, 20877,  1741,   497,    15,  1932,    93,     9,    32,
           18,   614,     2,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,  

In [ ]:
print(tokenized_dataset)

Dataset({
    features: ['label', 'text_cleaned', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1600000
})


In [ ]:
subset_tokenized_dataset = tokenized_dataset.shuffle(seed=39).select(range(20000))
print(subset_tokenized_dataset)

Dataset({
    features: ['label', 'text_cleaned', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 20000
})


# Finetune BERTweet

In [ ]:
# Split the subset of the tokenized dataset into train and test set with a 80/20 split
split_tokenized_dataset = subset_tokenized_dataset.train_test_split(test_size=0.2)
train_tokenized_dataset = split_tokenized_dataset['train']
test_tokenized_dataset = split_tokenized_dataset['test']

print(train_tokenized_dataset)
print(test_tokenized_dataset)

Dataset({
    features: ['label', 'text_cleaned', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 16000
})
Dataset({
    features: ['label', 'text_cleaned', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4000
})


In [ ]:
import os
from transformers import AutoModelForSequenceClassification

# Get the pretrained model - Bertweet
if os.path.exists(os.path.join(dir + 'bertweet/final_model/config.json')) and os.path.exists(os.path.join(dir + 'bertweet/final_model/model.safetensors')):
  print('Already fine-tuned!')
  model = AutoModelForSequenceClassification.from_pretrained(dir + 'bertweet/final_model', num_labels=3)
else:
  model = AutoModelForSequenceClassification.from_pretrained('vinai/bertweet-base', num_labels=3)

Already fine-tuned!


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# Define a metric to use during fine-tuning
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted')
    }

In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score

# Define the training arguments to finetune the pretrained model
training_args = TrainingArguments(
    output_dir=dir+'bertweet',
    run_name='Bertweet finetune',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir=dir+'bertweet/logs',
    learning_rate=3e-5
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=test_tokenized_dataset,
    compute_metrics=compute_metrics
)

# Start finetuning the pretrained model
trainer.train()

# Save the model after training
model.save_pretrained(dir+'bertweet/final_model')
tokenizer.save_pretrained(dir+'bertweet/tokenizer')

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.370900,0.352818,0.851000,0.851004
2,0.255700,0.403150,0.854250,0.854204
3,0.158900,0.520455,0.857750,0.857720


('/content/drive/My Drive/1513-project/bertweet/tokenizer/tokenizer_config.json',
 '/content/drive/My Drive/1513-project/bertweet/tokenizer/special_tokens_map.json',
 '/content/drive/My Drive/1513-project/bertweet/tokenizer/vocab.txt',
 '/content/drive/My Drive/1513-project/bertweet/tokenizer/bpe.codes',
 '/content/drive/My Drive/1513-project/bertweet/tokenizer/added_tokens.json')

# Preprocess Election Dataset

In [ ]:
import os
import glob
import pandas as pd
import datetime

In [ ]:
start_date = datetime.date(2024, 5, 1)
end_date = datetime.date(2024, 11, 30)
delta = datetime.timedelta(days=1)

files_to_download = []
current = start_date
while current <= end_date:
    files_to_download.append(current.strftime("%Y-%m-%d"))
    current += delta

print(files_to_download)

path_to_download = dir + 'election_data'

!mkdir -p '{path_to_download}'

final_csv_filename = 'combined_daily_dataset.csv'
if not os.path.exists(os.path.join(path_to_download, final_csv_filename)):
  dfs = []
  for file in files_to_download:
    url = f"https://github.com/YupengHuangFrank/ECE1513/raw/refs/heads/main/Date-Ordered-Data/{file}.csv.gz"
    filename = file + '.csv.gz'
    try:
      df = pd.read_csv(url, compression='gzip', usecols=['text', 'epoch'])
      n_samples = min(2000, len(df))
      sample_df = df.sample(n=n_samples, random_state=39)
      dfs.append(sample_df)
      print(f"Processed {filename}: sampled {n_samples} rows.")
    except Exception as e:
      print(f"Error processing csv {filename}: {e}")

  combined_df = pd.concat(dfs, ignore_index=True)
  # Save the combined DataFrame to csv for future load
  combined_file = os.path.join(path_to_download, final_csv_filename)
  combined_df.to_csv(combined_file, index=False)
  print(f"Combined csv saved at: {combined_file}")
else:
  print('Already combined, read from existing csv!')

print(combined_df)

['2024-05-01', '2024-05-02', '2024-05-03', '2024-05-04', '2024-05-05', '2024-05-06', '2024-05-07', '2024-05-08', '2024-05-09', '2024-05-10', '2024-05-11', '2024-05-12', '2024-05-13', '2024-05-14', '2024-05-15', '2024-05-16', '2024-05-17', '2024-05-18', '2024-05-19', '2024-05-20', '2024-05-21', '2024-05-22', '2024-05-23', '2024-05-24', '2024-05-25', '2024-05-26', '2024-05-27', '2024-05-28', '2024-05-29', '2024-05-30', '2024-05-31', '2024-06-01', '2024-06-02', '2024-06-03', '2024-06-04', '2024-06-05', '2024-06-06', '2024-06-07', '2024-06-08', '2024-06-09', '2024-06-10', '2024-06-11', '2024-06-12', '2024-06-13', '2024-06-14', '2024-06-15', '2024-06-16', '2024-06-17', '2024-06-18', '2024-06-19', '2024-06-20', '2024-06-21', '2024-06-22', '2024-06-23', '2024-06-24', '2024-06-25', '2024-06-26', '2024-06-27', '2024-06-28', '2024-06-29', '2024-06-30', '2024-07-01', '2024-07-02', '2024-07-03', '2024-07-04', '2024-07-05', '2024-07-06', '2024-07-07', '2024-07-08', '2024-07-09', '2024-07-10', '2024

In [ ]:
# Now read all files and combine into one dataframe for preprocessing
# all_files = glob.glob(os.path.join(path_to_download, "*.csv.gz"))
# print(all_files)

# if not os.path.exists(os.path.join(path_to_download, 'combined_dataset.csv')):
#   dfs = []
#   for file in all_files:
#       # Read each csv.gz file with pandas
#       print(file)
#       df = pd.read_csv(file, compression='gzip', usecols=['text', 'epoch'])
#       dfs.append(df)
#   combined_df = pd.concat(dfs, ignore_index=True)
#   combined_df.to_csv(os.path.join(path_to_download, 'combined_dataset.csv'), index=False)
# else:
#   print('Already combined, reading from existing csv')
#   combined_df = pd.read_csv(os.path.join(path_to_download, 'combined_dataset.csv'))

print(combined_df)
print(combined_df['text'].isnull().sum())

                                                     text         epoch
0       Trump rolls out the red carpet for foreign lea...  1.714602e+09
1       Donald Trump Takes Biggest Lead Ever over Joe ...  1.714604e+09
2       @PhonyLyingLibs @obriedave1 @DebraSc69769469 @...  1.714602e+09
3       Democrats: BAGS - Biden And Gullible Supporter...  1.714600e+09
4       @WhitneyChanell @AlmaSuono Not sure what there...  1.714595e+09
...                                                   ...           ...
369767  @truckdriverpleb He managed to outdo himself. ...  1.732961e+09
369768  @Benioff how much lower are twitter's revenues...  1.733004e+09
369769  @RangerUpCoffee Occam's razor.  the simplest e...  1.733006e+09
369770  Donald Trump’s pick for Treasury secretary has...  1.732990e+09
369771  @MdBreathe Republicans turned out all women to...  1.732949e+09

[369772 rows x 2 columns]
0
